<a href="https://colab.research.google.com/github/EoKaka120/TrabalhoDeLogica/blob/main/DesinvSist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DesinvSist

In [2]:
seucargo = (input("Qual o seu cargo?: "))

match seucargo:
  case "estagiario":
    print ("Estagiario, você tem acesso nas terças e quintas")
  case "analista":
    print ("analista, você tem acesso nas segundas quartas e sextas")
  case "gerente":
    print ("Ola Chefe, você tem acesso quando quiser")



Qual o seu cargo?: gerente
Ola Chefe, você tem acesso quando quiser


In [ ]:
idade = int(input("Qual sua idade?: "))
filme = input("Classificação do filme: ")

if idade <= 10 and filme == "Classificação PG":
    print("Você precisa estar acompanhado para assistir.")

match filme:
    case "Classificação G":
        print("Classificação G, você pode ver com qualquer idade, aproveite!")
    case "Classificação PG":
        if idade > 10:
            print("Classificação PG, você pode assistir sozinho.")
    case "Classificação R":
        print("Classificação R, você tem idade para assistir" if idade >= 18 else "Classificação R, você não pode assistir.")

Qual sua idade?: cu


ValueError: invalid literal for int() with base 10: 'cu'

In [ ]:
from IPython.core import application
from tkinter import *

root = Tk()

class application():
  def __init__(self):
    self.root = root
    self.tela()
    root.mainloop()

  def tela(self):
    self.root.title("Cadastro do cliente")
    self.root.configure(background="#5527b4") #cores no tkinter
    self.root.geometry("700c500")
    self.root.resizable(True, True)
    self.root.maxsize(width=900, height=700)
    self.root.minsize(width=400, height=300)

    # criar um loop para manter a janela aberta
    # root.mainloop()

In [ ]:
link = (input("Quantas pessoas entraram no link?: "))

match link:
  case "link1":
   print("Entraram 24 pessoas nesse link")
  case "link2":
   print("Entraram 12 pessoas nesse link")
  case "link3":
   print("entraram 6 pessoas nesse link")

Quantas pessoas entraram no link?: link 2


In [ ]:
link = float(input("Quantas pessoas estraram no Primeiro Link?: "))
link2 = link*2
link3 = link2*2

print(f"no 2° link entraram {link2}")
print(f"no 3° link entraram {link3}")

Quantas pessoas estraram no Primeiro Link?: 1
no 2° link entraram 2.0
no 3° link entraram 4.0


In [ ]:
nome = input("Qual é o seu nome? ")

if nome.strip().lower() == "bruno":
    print("Olá Bruno! Você é tchola🏳‍🌈💦")
else:
    print(f"Olá {nome} Você é incrivel 😄")

Qual é o seu nome? terzo
Olá Bruno! Você é tchola🏳‍🌈💦
Olá terzo você é tchola🏳‍🌈💦


In [ ]:
tabuada = 1
while tabuada <= 10:
  numero = 1
  while numero <= 10:
    print(f"{tabuada} x {numero} = {tabuada * numero}")
    numero += 1
tabuada += 1

# TesteDeCodigos

In [ ]:
"""
Como jogar:
  1) Salve este arquivo como rpg.py
  2) No terminal, rode:  python rpg.py
  3) Use os números/letras do menu para explorar, lutar, upar e salvar.

Recursos:
  - Sistema de combate por turnos com dano crítico, esquiva e habilidades.
  - Inventário (poções, armas) e ouro.
  - Mapa leve com encontros aleatórios, lojas e mini-missões.
  - Sistema de XP/nível com árvore simples de talentos.
  - Salvamento/Carregamento em JSON (savegame.json).
  - 100% biblioteca padrão (sem dependências externas).

Dica: personalize o JOGO_CONFIG abaixo para mudar a dificuldade e a frequência de encontros.
"""
from __future__ import annotations
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Optional
import json
import os
import random
import textwrap

# ==============================
# CONFIGURAÇÕES DO JOGO
# ==============================
JOGO_CONFIG = {
    "crit_chance_base": 0.10,     # 10% base de crítico
    "dodge_chance_base": 0.07,    # 7% base de esquiva
    "loot_chance": 0.60,          # chance de drop ao vencer
    "encounter_chance": 0.55,     # chance de encontro ao explorar
    "shop_chance": 0.20,          # chance de encontrar uma loja ao explorar
    "mission_chance": 0.25,       # chance de missão curta ao explorar
    "hp_per_level": 12,           # vida ganha por nível
    "xp_table": [20, 45, 80, 120, 180, 260, 360, 500],  # XP p/ níveis 2..9
    "max_level": 10,
}

# Utilitário de impressão com quebra bonita
wrap = lambda s: print(textwrap.fill(s, width=78))

# ==============================
# ITENS
# ==============================
@dataclass
class Item:
    nome: str
    preco: int
    descricao: str

@dataclass
class Pocao(Item):
    cura: int = 20

@dataclass
class Arma(Item):
    ataque_bonus: int = 1

ITEMS_PADRAO: Dict[str, Item] = {
    "Poção Pequena": Pocao("Poção Pequena", 10, "Cura 20 HP", cura=20),
    "Poção Média":   Pocao("Poção Média",   25, "Cura 45 HP", cura=45),
    "Poção Grande":  Pocao("Poção Grande",  50, "Cura 90 HP", cura=90),
    "Adaga":         Arma("Adaga",          20, "+1 ATK", ataque_bonus=1),
    "Espada Curta":  Arma("Espada Curta",   45, "+3 ATK", ataque_bonus=3),
    "Espada Longa":  Arma("Espada Longa",   90, "+6 ATK", ataque_bonus=6),
}

# ==============================
# PERSONAGENS
# ==============================
@dataclass
class Talentos:
    feroz: int = 0   # +crit chance por ponto
    agil: int = 0    # +dodge por ponto
    vital: int = 0   # +HP por ponto

    def aplicar(self, jogador: "Jogador") -> None:
        jogador.crit_chance += 0.02 * self.feroz
        jogador.dodge_chance += 0.02 * self.agil
        jogador.max_hp += 5 * self.vital

@dataclass
class Entidade:
    nome: str
    nivel: int
    max_hp: int
    ataque: int
    defesa: int
    hp: int = field(init=False)
    crit_chance: float = 0.0
    dodge_chance: float = 0.0

    def __post_init__(self):
        self.hp = self.max_hp

    def esta_vivo(self) -> bool:
        return self.hp > 0

    def receber_dano(self, dmg: int) -> int:
        self.hp = max(0, self.hp - dmg)
        return dmg

@dataclass
class Jogador(Entidade):
    xp: int = 0
    ouro: int = 30
    talentos: Talentos = field(default_factory=Talentos)
    inventario: Dict[str, int] = field(default_factory=lambda: {"Poção Pequena": 2})
    arma: Optional[Arma] = None

    def poder_de_ataque(self) -> int:
        bonus = self.arma.ataque_bonus if self.arma else 0
        return self.ataque + bonus

    def ganhar_xp(self, qtd: int) -> None:
        self.xp += qtd
        while self.nivel < JOGO_CONFIG["max_level"] and self.xp >= proximo_xp(self.nivel):
            self.nivel += 1
            ganho = JOGO_CONFIG["hp_per_level"]
            self.max_hp += ganho
            self.hp = self.max_hp
            wrap(f"\nVocê subiu para o nível {self.nivel}! +{ganho} HP máximo.")
            escolher_talento(self)

@dataclass
class Inimigo(Entidade):
    xp_recompensa: int = 10
    ouro_recompensa: int = 8

# ==============================
# MECÂNICAS
# ==============================

def proximo_xp(nivel_atual: int) -> int:
    if nivel_atual <= 1:
        return JOGO_CONFIG["xp_table"][0]
    idx = min(nivel_atual-1, len(JOGO_CONFIG["xp_table"])) - 1
    if idx < len(JOGO_CONFIG["xp_table"]):
        return JOGO_CONFIG["xp_table"][idx]
    # níveis além da tabela — cresce linearmente
    return JOGO_CONFIG["xp_table"][-1] + 100 * (nivel_atual - len(JOGO_CONFIG["xp_table"]))


def rolar_crit(chance_extra: float = 0.0) -> bool:
    return random.random() < (JOGO_CONFIG["crit_chance_base"] + chance_extra)


def rolar_esquiva(entidade: Entidade) -> bool:
    return random.random() < (JOGO_CONFIG["dodge_chance_base"] + entidade.dodge_chance)


def calcular_dano(atacante: Entidade, defensor: Entidade) -> (int, bool):
    base = max(1, atacante.ataque - defensor.defesa)
    variacao = random.randint(-2, 2)
    dano = max(1, base + variacao)
    crit = rolar_crit(atacante.crit_chance)
    if crit:
        dano = int(dano * 1.8)
    return dano, crit


def acao_atacar(atacante: Entidade, defensor: Entidade) -> str:
    if rolar_esquiva(defensor):
        return f"{defensor.nome} esquivou!"
    dano, crit = calcular_dano(atacante, defensor)
    defensor.receber_dano(dano)
    if crit:
        return f"Acerto CRÍTICO! {atacante.nome} causou {dano} de dano em {defensor.nome}."
    return f"{atacante.nome} causou {dano} de dano em {defensor.nome}."


def usar_pocao(j: Jogador) -> str:
    opcoes = [k for k, v in j.inventario.items() if v > 0 and isinstance(ITEMS_PADRAO.get(k), Pocao)]
    if not opcoes:
        return "Você não tem poções."
    print("\nPoções no inventário:")
    for i, nome in enumerate(opcoes, 1):
        p: Pocao = ITEMS_PADRAO[nome]  # type: ignore
        print(f"  [{i}] {nome} (+{p.cura} HP) — {j.inventario[nome]}x")
    escolha = input("Selecione uma poção (ou Enter p/ voltar): ").strip()
    if not escolha.isdigit():
        return "Cancelado."
    idx = int(escolha) - 1
    if idx < 0 or idx >= len(opcoes):
        return "Opção inválida."
    nome = opcoes[idx]
    p: Pocao = ITEMS_PADRAO[nome]  # type: ignore
    j.inventario[nome] -= 1
    antes = j.hp
    j.hp = min(j.max_hp, j.hp + p.cura)
    return f"Você usou {nome} e curou {j.hp - antes} HP."


def escolher_talento(j: Jogador) -> None:
    wrap("\nEscolha um talento:")
    print("  [1] Feroz (+2% crit por ponto)")
    print("  [2] Ágil  (+2% esquiva por ponto)")
    print("  [3] Vital (+5 HP máx por ponto)")
    esc = input("> ").strip()
    if esc == "1":
        j.talentos.feroz += 1
    elif esc == "2":
        j.talentos.agil += 1
    else:
        j.talentos.vital += 1
    # Reaplicar talentos
    j.crit_chance = 0.0
    j.dodge_chance = 0.0
    j.talentos.aplicar(j)
    wrap("Talento aplicado!\n")

# ==============================
# ENCONTROS / GERADORES
# ==============================
NOMES_INIMIGOS = [
    "Goblin", "Lobo Cinzento", "Bandido", "Aranha das Cavernas",
    "Esqueleto", "Cultista", "Slime", "Sombra"
]


def gerar_inimigo(nivel: int) -> Inimigo:
    nome = random.choice(NOMES_INIMIGOS)
    escalar = 1 + (nivel - 1) * 0.2
    max_hp = int(random.randint(18, 28) * escalar)
    atk = int(random.randint(6, 10) * escalar)
    dfs = int(random.randint(1, 5) * escalar)
    inim = Inimigo(nome=nome, nivel=nivel, max_hp=max_hp, ataque=atk, defesa=dfs,
                   xp_recompensa=random.randint(10, 20) + nivel * 3,
                   ouro_recompensa=random.randint(5, 12) + nivel * 2)
    inim.crit_chance = 0.03
    inim.dodge_chance = 0.03
    return inim


def encontrar_loja(j: Jogador) -> None:
    wrap("Você encontrou um mercador viajante!")
    itens = [ITEMS_PADRAO["Poção Pequena"], ITEMS_PADRAO["Poção Média"], ITEMS_PADRAO["Poção Grande"],
             ITEMS_PADRAO["Adaga"], ITEMS_PADRAO["Espada Curta"], ITEMS_PADRAO["Espada Longa"]]
    while True:
        print(f"\nSeu ouro: {j.ouro}")
        for i, it in enumerate(itens, 1):
            extra = ""
            if isinstance(it, Arma):
                extra = f" (+{it.ataque_bonus} ATK)"
            elif isinstance(it, Pocao):
                extra = f" (+{it.cura} HP)"
            print(f"  [{i}] {it.nome} — {it.preco} ouro{extra}")
        print("  [0] Sair da loja")
        e = input("> ").strip()
        if e == "0" or e == "":
            break
        if not e.isdigit() or not (1 <= int(e) <= len(itens)):
            wrap("Escolha inválida.")
            continue
        it = itens[int(e)-1]
        if j.ouro < it.preco:
            wrap("Você não tem ouro suficiente.")
            continue
        j.ouro -= it.preco
        if isinstance(it, Pocao):
            j.inventario[it.nome] = j.inventario.get(it.nome, 0) + 1
            wrap(f"Comprou {it.nome}!")
        else:
            j.arma = it
            wrap(f"Você equipou {it.nome}!")


def mini_missao(j: Jogador) -> None:
    wrap("Um aldeão pede ajuda para espantar criaturas próximas...")
    rodadas = random.randint(2, 4)
    total_xp = 0
    total_ouro = 0
    for r in range(rodadas):
        inim = gerar_inimigo(max(1, j.nivel + random.randint(-1, 1)))
        wrap(f"Onda {r+1}/{rodadas}: {inim.nome} (Nv {inim.nivel}) aparece!")
        combate(j, inim)
        if not j.esta_vivo():
            wrap("Você desmaiou no meio da missão!")
            return
        total_xp += inim.xp_recompensa
        total_ouro += inim.ouro_recompensa
    j.ganhar_xp(total_xp)
    j.ouro += total_ouro
    wrap(f"Missão concluída! Recompensas: +{total_xp} XP, +{total_ouro} ouro.")

# ==============================
# COMBATE
# ==============================

def mostrar_status(j: Jogador, inim: Inimigo) -> None:
    print("\n================ COMBATE ================")
    print(f"Você: HP {j.hp}/{j.max_hp} | ATK {j.poder_de_ataque()} | DEF {j.defesa}")
    arma = j.arma.nome if j.arma else "(sem arma)"
    print(f"Arma: {arma} | Crit ~{int((JOGO_CONFIG['crit_chance_base']+j.crit_chance)*100)}%  Esquiva ~{int((JOGO_CONFIG['dodge_chance_base']+j.dodge_chance)*100)}%")
    print(f"Inimigo: {inim.nome} Nv {inim.nivel} — HP {inim.hp}/{inim.max_hp} | ATK {inim.ataque} | DEF {inim.defesa}")
    print("========================================")


def combate(j: Jogador, inim: Inimigo) -> None:
    while j.esta_vivo() and inim.esta_vivo():
        mostrar_status(j, inim)
        print("[A] Atacar   [P] Poção   [F] Fugir")
        esc = input("> ").strip().lower()
        if esc == "a":
            msg = acao_atacar(j, inim)
            wrap(msg)
        elif esc == "p":
            wrap(usar_pocao(j))
        elif esc == "f":
            if random.random() < 0.50:
                wrap("Você fugiu com sucesso!")
                return
            else:
                wrap("Falhou em fugir!")
        else:
            wrap("Ação inválida.")
            continue

        # Turno do inimigo se ele estiver vivo
        if inim.esta_vivo():
            msg = acao_atacar(inim, j)
            wrap(msg)
        else:
            wrap(f"Você derrotou {inim.nome}!")
            if random.random() < JOGO_CONFIG["loot_chance"]:
                loot_aleatorio(j)
            j.ganhar_xp(inim.xp_recompensa)
            j.ouro += inim.ouro_recompensa
            wrap(f"Recompensas: +{inim.xp_recompensa} XP, +{inim.ouro_recompensa} ouro.")

    if not j.esta_vivo():
        wrap("Você caiu em batalha... mas acorda na cidade com 1 HP e perde metade do ouro!")
        j.hp = 1
        j.ouro //= 2


def loot_aleatorio(j: Jogador) -> None:
    possiveis = ["Poção Pequena", "Poção Média", "Adaga"]
    nome = random.choice(possiveis)
    j.inventario[nome] = j.inventario.get(nome, 0) + 1
    wrap(f"\nVocê encontrou um item: {nome}!")

# ==============================
# SALVAMENTO
# ==============================
SAVE_ARQ = "savegame.json"


def salvar_jogo(j: Jogador) -> None:
    dados = asdict(j)
    # Arma vira apenas o nome
    if j.arma:
        dados["arma"] = j.arma.nome
    with open(SAVE_ARQ, "w", encoding="utf-8") as f:
        json.dump(dados, f, ensure_ascii=False, indent=2)
    wrap(f"Jogo salvo em {SAVE_ARQ}!")


def carregar_jogo() -> Optional[Jogador]:
    if not os.path.exists(SAVE_ARQ):
        return None
    try:
        with open(SAVE_ARQ, "r", encoding="utf-8") as f:
            d = json.load(f)
        j = Jogador(
            nome=d["nome"], nivel=d["nivel"], max_hp=d["max_hp"], ataque=d["ataque"], defesa=d["defesa"],
            xp=d.get("xp", 0), ouro=d.get("ouro", 0),
        )
        j.hp = d.get("hp", j.max_hp)
        j.crit_chance = d.get("crit_chance", 0.0)
        j.dodge_chance = d.get("dodge_chance", 0.0)
        t = d.get("talentos", {})
        j.talentos = Talentos(**{k:int(v) for k,v in t.items()})
        j.inventario = {k:int(v) for k,v in d.get("inventario", {}).items()}
        arma_nome = d.get("arma")
        if arma_nome and arma_nome in ITEMS_PADRAO and isinstance(ITEMS_PADRAO[arma_nome], Arma):
            j.arma = ITEMS_PADRAO[arma_nome]  # type: ignore
        j.talentos.aplicar(j)
        return j
    except Exception as e:
        wrap(f"Falha ao carregar: {e}")
        return None

# ==============================
# LOOP PRINCIPAL
# ==============================

def criar_jogador() -> Jogador:
    wrap("Bem-vindo a Elaria, viajante! Como você se chama?")
    nome = input("> ").strip() or "Herói"
    j = Jogador(nome=nome, nivel=1, max_hp=40, ataque=8, defesa=3)
    j.talentos.aplicar(j)
    wrap(f"Saudações, {j.nome}! Sua aventura começa agora.")
    return j


def status_completo(j: Jogador) -> None:
    arma = j.arma.nome if j.arma else "(sem)"
    wrap(
        f"{j.nome} — Nv {j.nivel}  XP:{j.xp}/{proximo_xp(j.nivel)}  HP:{j.hp}/{j.max_hp}\n"
        f"ATK:{j.poder_de_ataque()}  DEF:{j.defesa}  Ouro:{j.ouro}  Arma:{arma}\n"
        f"Talentos → Feroz:{j.talentos.feroz}  Ágil:{j.talentos.agil}  Vital:{j.talentos.vital}"
    )


def explorar(j: Jogador) -> None:
    wrap("Você explora os arredores...")
    r = random.random()
    if r < JOGO_CONFIG["encounter_chance"]:
        inim = gerar_inimigo(max(1, j.nivel + random.randint(-1, 1)))
        wrap(f"Um {inim.nome} aparece!")
        combate(j, inim)
    elif r < JOGO_CONFIG["encounter_chance"] + JOGO_CONFIG["shop_chance"]:
        encontrar_loja(j)
    elif r < JOGO_CONFIG["encounter_chance"] + JOGO_CONFIG["shop_chance"] + JOGO_CONFIG["mission_chance"]:
        mini_missao(j)
    else:
        wrap("Nada acontece. Um momento de paz...")


def menu_principal(j: Jogador) -> None:
    while True:
        print("\n===== CIDADE DE Elaria =====")
        print("[1] Explorar")
        print("[2] Status")
        print("[3] Usar Poção")
        print("[4] Salvar Jogo")
        print("[5] Carregar Jogo")
        print("[0] Sair")
        esc = input("> ").strip()
        if esc == "1":
            explorar(j)
        elif esc == "2":
            status_completo(j)
        elif esc == "3":
            wrap(usar_pocao(j))
        elif esc == "4":
            salvar_jogo(j)
        elif esc == "5":
            j2 = carregar_jogo()
            if j2:
                j.__dict__.update(j2.__dict__)
                wrap("Jogo carregado!")
            else:
                wrap("Nenhum salvamento encontrado.")
        elif esc == "0":
            wrap("Até a próxima, aventureiro!")
            break
        else:
            wrap("Opção inválida.")


def main():
    j = carregar_jogo()
    if j:
        wrap(f"Bem-vindo de volta, {j.nome}! A aventura continua...")
    else:
        j = criar_jogador()
    menu_principal(j)

if __name__ == "__main__":
    random.seed()  # inicializa com fonte de entropia do sistema
    main()

Bem-vindo a Elaria, viajante! Como você se chama?
Saudações, 20Comer! Sua aventura começa agora.

===== CIDADE DE Elaria =====
[1] Explorar
[2] Status
[3] Usar Poção
[4] Salvar Jogo
[5] Carregar Jogo
[0] Sair
20Comer — Nv 1  XP:0/20  HP:40/40 ATK:8  DEF:3  Ouro:30  Arma:(sem) Talentos →
Feroz:0  Ágil:0  Vital:0

===== CIDADE DE Elaria =====
[1] Explorar
[2] Status
[3] Usar Poção
[4] Salvar Jogo
[5] Carregar Jogo
[0] Sair
Você explora os arredores...
Um aldeão pede ajuda para espantar criaturas próximas...
Onda 1/4: Aranha das Cavernas (Nv 1) aparece!

================ COMBATE ================
Você: HP 40/40 | ATK 8 | DEF 3
Arma: (sem arma) | Crit ~10%  Esquiva ~7%
Inimigo: Aranha das Cavernas Nv 1 — HP 26/26 | ATK 10 | DEF 1
[A] Atacar   [P] Poção   [F] Fugir
20Comer causou 5 de dano em Aranha das Cavernas.
20Comer esquivou!

================ COMBATE ================
Você: HP 40/40 | ATK 8 | DEF 3
Arma: (sem arma) | Crit ~10%  Esquiva ~7%
Inimigo: Aranha das Cavernas Nv 1 — HP 21/26 | 